In [ ]:
from MOCA.MOCA_GeneratedCode.src.TurningOptimization import TurningOptimization
from MOCA.MOCA_GeneratedCode.src.MillingOptimization import MillingOptimization

import simpy

# Turning process time calculation
# turning = TurningOptimization()
# turning.run()
# turning_time = turning['TurningGroup_inst.cutting_time']
turning_time = 1.94

# Milling process time calculation
# milling = MillingOptimization()
# milling.run()
# milling_time = milling['MillingGroupInstance.MachiningTime_Tm']
milling_time = 1.72

turning_produce_flag = False
milling_produce_flag = False


def producer_turning(simpy_env, tm_buffer):
    while True:
        if turning_produce_flag:
            # Simulate turning production
            yield simpy_env.timeout(turning_time)

            # Put the product in the buffer
            # This can be a blocking call depending upon buffer situation
            yield tm_buffer.put()


def consumer_milling(simpy_env, tm_buffer):
    while True:
        if milling_produce_flag:
            # Simulate milling production
            yield simpy_env.timeout(milling_time)

            # Get the product from the buffer
            # This can also be a blocking call depending upon buffer situation
            yield tm_buffer.get()


CONFIG_TURNING_ON_TIME = 480            # in minutes i.e. 8 hrs
CONFIG_TURNING_PERIOD = 1440            # in minutes i.e. 24 hrs


def turning_controller(simpy_env):
    while True:
        turning_produce_flag = True     # Start turning production
        yield simpy_env.timeout(CONFIG_TURNING_ON_TIME)
        turning_produce_flag = False    # Stop turning production
        yield simpy_env.timeout(CONFIG_TURNING_PERIOD - CONFIG_TURNING_ON_TIME)


CONFIG_MILLING_ON_TIME = 540            # 9 hrs
CONFIG_MILLING_PERIOD = 1440            # 24 hrs


def milling_controller(simpy_env):
    while True:
        milling_produce_flag = True     # Start turning production
        yield simpy_env.timeout(CONFIG_MILLING_ON_TIME)
        milling_produce_flag = False    # Stop turning production
        yield simpy_env.timeout(CONFIG_MILLING_PERIOD - CONFIG_MILLING_ON_TIME)


buffer_states = []


def buffer_monitor(simpy_env, tm_buffer):
    while True:
        yield simpy_env.timeout(0.1)
        buffer_states.append(len(tm_buffer.items))


if __name__ == "__main__":
    ####################
    # SimPy simulation #
    ####################

    # SimPy simulation environment (an event dispatcher and task scheduler)
    env = simpy.Environment()

    # Buffer of 10 capacity
    turning_milling_buffer = simpy.Store(env, capacity=100)

    prod = env.process(producer_turning(env, turning_milling_buffer))
    cons = env.process(consumer_milling(env, turning_milling_buffer))
    prod_ctrl = env.process(turning_controller(env))
    cons_ctrl = env.process(milling_controller(env))
    buff_mon = env.process(buffer_monitor(env, turning_milling_buffer))

    env.run(until=7200)

    print buffer_states[0:1000]
    print "finished"
